In [ ]:
!pip install speechrecognition gtts deep-translator rank-bm25 sentence-transformers playsound pickle-mixin


In [57]:
pip install pygame



   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.6 MB 670.4 kB/s eta 0:00:16
   - -------------------------------------- 0.5/10.6 MB 670.4 kB/s eta 0:00:16
   --- ------------------------------------ 1.0/10.6 MB 914.5 kB/s eta 0:00:11
   ---- ----------------------------------- 1.3/10.6 MB 986.4 kB/s eta 0:00:10
   ----- ---------------------------------- 1.6/10.6 MB 1.1 MB/s eta 0:00:09
   -------- ------------------------------- 2.4/10.6 MB 1.4 MB/s eta 0:00:06
   --------- ------------------------------ 2.6/10.6 MB 1.5 MB/s eta 0:00:06
   ------------- -------------------------- 3.7/10.6 MB 1.8 MB/s eta 0:00:04
   ---------------- ----------------------- 4.5/10.6 MB 2.0 MB/s eta 0:00:04
   ---------------

In [39]:
import speech_recognition as sr
import time
import pandas as pd
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
from deep_translator import GoogleTranslator
from gtts import gTTS
from playsound import playsound
import pickle

# Function to provide progress updates
def update_checkpoint(message):
    print(f"[{time.strftime('%H:%M:%S')}] {message}")


In [40]:
def build_or_load_bm25(dataset_path, cache_path="bm25_cache.pkl"):
    try:
        # Try loading cached BM25 model and tokenized corpus
        with open(cache_path, "rb") as f:
            bm25, df = pickle.load(f)
        update_checkpoint("Loaded BM25 model from cache.")
    except FileNotFoundError:
        # If no cache is found, build BM25 from scratch and save it
        update_checkpoint("Building BM25 model from scratch...")
        df = pd.read_csv(dataset_path, low_memory=False)
        df['Answer'] = df['Answer'].astype(str).fillna("")
        tokenized_corpus = [doc.split(" ") for doc in df['Answer'] if isinstance(doc, str)]
        bm25 = BM25Okapi(tokenized_corpus)
        # Cache the BM25 model and tokenized dataset for future use
        with open(cache_path, "wb") as f:
            pickle.dump((bm25, df), f)
        update_checkpoint("BM25 model built and saved to cache.")
    return bm25, df


In [41]:
translation_cache = {}

def translate_text(text):
    if text in translation_cache:
        update_checkpoint("Using cached translation.")
        return translation_cache[text]
    try:
        update_checkpoint("Translating text from Hindi to English...")
        translated_text = GoogleTranslator(source='hi', target='en').translate(text)
        translation_cache[text] = translated_text
        update_checkpoint(f"Translation successful: '{translated_text}'")
        return translated_text
    except Exception as e:
        update_checkpoint(f"Translation error: {str(e)}")
        return None


In [43]:
translation_cache = {}

def translate_text(text):
    if text in translation_cache:
        update_checkpoint("Using cached translation.")
        return translation_cache[text]
    try:
        update_checkpoint("Translating text from Hindi to English...")
        translated_text = GoogleTranslator(source='hi', target='en').translate(text)
        translation_cache[text] = translated_text
        update_checkpoint(f"Translation successful: '{translated_text}'")
        return translated_text
    except Exception as e:
        update_checkpoint(f"Translation error: {str(e)}")
        return None


In [45]:
from transformers import pipeline

# Global variable to cache the QA pipeline
qa_pipeline = None

def load_qa_model():
    global qa_pipeline
    if qa_pipeline is None:
        update_checkpoint("Loading Question-Answering model...")
        qa_pipeline = pipeline("question-answering", model="deepset/bert-base-cased-squad2")
        update_checkpoint("Question-Answering model loaded.")
    return qa_pipeline


In [46]:
def capture_audio():
    r = sr.Recognizer()

    try:
        update_checkpoint("Starting audio capture...")
        with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source)
            print("Speak something...")
            audio = r.listen(source)
        update_checkpoint("Audio capture successful.")

        # Recognize speech using Google Speech Recognition
        update_checkpoint("Converting speech to text...")
        text = r.recognize_google(audio)
        update_checkpoint(f"Speech recognized: '{text}'")
        return text
    except sr.UnknownValueError:
        update_checkpoint("Speech recognition error: Could not understand audio.")
        return None
    except sr.RequestError:
        update_checkpoint("Speech recognition service unavailable.")
        return None
    except Exception as e:
        update_checkpoint(f"Unexpected error: {str(e)}")
        return None


In [47]:
def retrieve_documents(query, bm25, df):
    try:
        # Tokenize the query and perform BM25 search
        update_checkpoint("Performing BM25 retrieval...")
        query_tokens = query.split(" ")
        results = bm25.get_top_n(query_tokens, df['Answer'], n=5)
        update_checkpoint("BM25 retrieval successful.")
        return results
    except Exception as e:
        update_checkpoint(f"Retrieval error: {str(e)}")
        return None


In [48]:
def answer_question(query, context):
    try:
        update_checkpoint("Performing question-answering...")
        qa_pipeline = load_qa_model()
        result = qa_pipeline(question=query, context=context[0])  # Use the first document for QA
        update_checkpoint("Question-answering successful.")
        return result['answer']
    except Exception as e:
        update_checkpoint(f"QA error: {str(e)}")
        return None


In [58]:
import pygame

def play_audio_with_pygame(filename):
    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()

    # Wait for the audio to finish playing
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

def text_to_speech(text, filename="output.mp3"):
    try:
        update_checkpoint("Converting answer to speech...")
        speech = gTTS(text=text, lang='en', slow=False)
        speech.save(filename)
        update_checkpoint(f"Speech saved as '{filename}'.")

        # Play the audio using Pygame
        play_audio_with_pygame(filename)

    except Exception as e:
        update_checkpoint(f"TTS error: {str(e)}")


pygame 2.6.1 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [59]:
def main_pipeline():
    # Step 1: Capture Audio
    query_text = capture_audio()

    if query_text:
        # Step 2: Translate Text (if needed, based on language)
        translated_query = translate_text(query_text)

        if translated_query:
            # Step 3: Retrieve Relevant Documents
            bm25, df = build_or_load_bm25("data\\new_csv_file.csv")
            relevant_docs = retrieve_documents(translated_query, bm25, df)

            if relevant_docs:
                # Step 4: Answer Question using retrieved documents
                answer = answer_question(translated_query, relevant_docs)

                if answer:
                    # Step 5: Convert Answer to Speech
                    text_to_speech(answer)

# Run the full pipeline
main_pipeline()


[00:23:11] Starting audio capture...
Speak something...
[00:23:23] Audio capture successful.
[00:23:23] Converting speech to text...
[00:23:26] Speech recognized: 'Kuchh BNP shulk Vaid ho sakte hain lekin parinaam theek hai sanyukt rashtra'
[00:23:26] Translating text from Hindi to English...
[00:23:27] Translation successful: 'Some BNP charges may be valid but the result is good United Nation'
[00:23:45] Loaded BM25 model from cache.
[00:23:45] Performing BM25 retrieval...
[00:23:49] BM25 retrieval successful.
[00:23:49] Performing question-answering...
[00:23:49] Question-answering successful.
[00:23:49] Converting answer to speech...
[00:23:50] Speech saved as 'output.mp3'.
